In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[4],3b
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,0
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0]"
ng_refs,-,"[5, 4]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312        1  66.811869                    1  44.714254
0.000750        2  30.192711                    2  32.076582
0.001052        3  28.255311                    3  31.058694
0.001476        4  25.341714                    4  28.380811
0.002070        5  22.174221                    5  24.342332
0.002904        6  19.226535                    6  19.784353
0.004074        7  16.722187                    7  15.804571
0.005714        8  14.724929                    8  13.239320
0.008015        9  13.199432                    9  11.985891
0.011243       10  12.055478                   10  11.364303
0.015771       11  11.189547                   11  10.773306
0.022122       12  10.518709                   12  10.025188
0.031031       13  10.089954                   13   9.372094
0.043528       14   9.446098                   14   8.711925
0.061057       15   8.347313                   15   7.987662
0.085645       16   7.340092                   16   7.552884
0.120136       17   6.662366                   17   7.298054
0.168516       18   6.248298                   18   6.955353
0.236378       19   6.084259                   19   6.519815
0.331549       20   6.196347                   20   6.213776
0.465100       21   6.809821                   21   6.506363
0.652400       22   6.510550                   22   6.208329
0.915100       23   5.072440                   23   4.971479
1.283650       24   3.910509                   24   3.886141
1.800600       25   3.126184                   25   3.010487
2.525700       26   2.505188                   26   2.337712
3.542800       27   1.954727                   27   1.948574
4.969550       28   1.571513                   28   1.699260
6.970850       29   1.401774                   29   1.507472
9.778100       30   1.262337                   30   1.299257
13.715850      31   1.074496                   31   1.104575
19.239350      32   0.841936                   32   0.829572
26.987250      33   0.698846                   33   0.668487
37.855300      34   0.648636                   34   0.677182
53.100050      35   0.551658                   35   0.600669
73.887500      36   0.434397                   36   0.431625
97.662500      37   0.339799                   37   0.316659
121.437500     38   0.247667                   38   0.250845
145.212500     39   0.170539                   39   0.198201
168.987500     40   0.126622                   40   0.161723
192.762500     41   0.095783                   41   0.125035
216.537500     42   0.070982                   42   0.088939
240.312500     43   0.047261                   43   0.052509
264.087500     44   0.012151                   44   0.006720
287.862500     45  -0.067821                   45  -0.081792
311.637500     46  -0.102537                   46  -0.120709
335.412500     47  -0.036092                   47  -0.054391
359.187500     48   0.019780                   48   0.004912
382.962500     49   0.036545                   49   0.025602
406.737500     50   0.044235                   50   0.036242
430.512500     51   0.048698                   51   0.042436
454.287500     52   0.051478                   52   0.045171
478.062500     53   0.053335                   53   0.048028
501.837500     54   0.054652                   54   0.049812
525.612500     55   0.055710                   55   0.051708
549.387500     56   0.056489                   56   0.053210
573.162500     57   0.056954                   57   0.054356
596.937500     58   0.056909                   58   0.055086
620.712500     59   0.056070                   59   0.054103
644.487500     60   0.054315                   60   0.053440
668.262500     61   0.051681                   61   0.050742
692.037500     62   0.048408                   62   0.047311
715.812500     63   0.044866                   63

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -14.290363   0.000000 -14.290363 -14.145341 -1.336097e-07   
0.000624    2     -14.291621   0.006200 -14.285421 -14.146583  4.549109e-03   
0.000876    3     -14.292224   0.007703 -14.284522 -14.147119  6.040773e-03   
0.001229    4     -14.292966   0.009625 -14.283341 -14.147788  8.006984e-03   
0.001723    5     -14.293870   0.012014 -14.281856 -14.148613  1.049614e-02   
0.002417    6     -14.294965   0.014932 -14.280033 -14.149624  1.350805e-02   
0.003391    7     -14.296284   0.018469 -14.277816 -14.150851  1.701745e-02   
0.004757    8     -14.297864   0.022754 -14.275111 -14.152333  2.105621e-02   
0.006672    9     -14.299741   0.027972 -14.271769 -14.154105  2.583292e-02   
0.009359    10    -14.301953   0.034385 -14.267568 -14.156204  3.174693e-02   
0.013128    11    -14.304530   0.042344 -14.262186 -14.158657  3.927434e-02   
0.018415    12    -14.307484   0.052305 -14.255179 -14.161473  4.883768e-02   
0.025830    13    -14.310774   0.064835 -14.245939 -14.164606  6.077744e-02   
0.036232    14    -14.314111   0.080606 -14.233506 -14.167784  7.550543e-02   
0.050823    15    -14.317192   0.100013 -14.217179 -14.170738  9.351871e-02   
0.071291    16    -14.320175   0.123234 -14.196941 -14.173606  1.157550e-01   
0.100000    17    -14.323171   0.151192 -14.171978 -14.176465  1.443026e-01   
0.140271    18    -14.325843   0.185646 -14.140197 -14.178958  1.816135e-01   
0.196760    19    -14.327477   0.229092 -14.098386 -14.180341  2.295433e-01   
0.275997    20    -14.326558   0.285280 -14.041278 -14.179095  2.894999e-01   
0.387100    21    -14.319311   0.359583 -13.959728 -14.171494  3.636869e-01   
0.543100    22    -14.292482   0.458595 -13.833887 -14.145145  4.575826e-01   
0.761700    23    -14.238761   0.573463 -13.665298 -14.093050  5.662681e-01   
1.068500    24    -14.170057   0.689104 -13.480952 -14.026438  6.803508e-01   
1.498800    25    -14.092345   0.810720 -13.281626 -13.951039  8.030573e-01   
2.102400    26    -14.004916   0.946815 -13.058101 -13.865600  9.328925e-01   
2.949000    27    -13.908836   1.101970 -12.806866 -13.770961  1.072717e+00   
4.136600    28    -13.814071   1.282195 -12.531876 -13.675870  1.251779e+00   
5.802500    29    -13.730627   1.508870 -12.221757 -13.591304  1.502576e+00   
8.139200    30    -13.651048   1.817300 -11.833748 -13.509517  1.838099e+00   
11.417000   31    -13.570396   2.226787 -11.343610 -13.425476  2.258584e+00   
16.014700   32    -13.503915   2.745508 -10.758406 -13.355103  2.789858e+00   
22.464000   33    -13.505078   3.389882 -10.115196 -13.355045  3.423630e+00   
31.510500   34    -13.601566   4.235268  -9.366298 -13.453280  4.238304e+00   
44.200100   35    -13.746123   5.354837  -8.391285 -13.599048  5.402098e+00   
62.000000   36    -13.916374   6.688273  -7.228101 -13.768292  6.837999e+00   
85.775000   37    -14.071263   8.066562  -6.004701 -13.921274  8.206699e+00   
109.550000  38    -14.143895   9.096176  -5.047719 -13.989741  9.167070e+00   
133.325000  39    -14.172398   9.822188  -4.350210 -14.011545  9.895405e+00   
157.100000  40    -14.205691  10.335774  -3.869917 -14.037127  1.047924e+01   
180.875000  41    -14.247556  10.734246  -3.513310 -14.070581  1.096821e+01   
204.650000  42    -14.300698  11.057142  -3.243556 -14.115326  1.136513e+01   
228.425000  43    -14.369705  11.326057  -3.043647 -14.177775  1.167808e+01   
252.200000  44    -14.464682  11.554138  -2.910545 -14.269900  1.191810e+01   
275.975000  45    -14.627340  11.751017  -2.876323 -14.434846  1.210198e+01   
299.750000  46    -15.017750  11.950421  -3.067329 -14.835761  1.227252e+01   
323.525000  47    -15.688954  12.332850  -3.356104 -15.523120  1.261989e+01   
347.300000  48    -16.469042  13.011291  -3.457751 -16.321573  1.326514e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')